In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import openpyxl
import plotly.express as px

d:\Descargas\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class Data:
    def __init__(self, base_url='https://wdc.kugi.kyoto-u.ac.jp/dst_final/'):
        self.base_url = base_url

    def get_data_by_month(self, year, month):
        if(int(month)>9):
            url = f'{self.base_url}{year}{month}/index.html'
        else:
            url = f'{self.base_url}{year}0{month}/index.html'
        return url
    
    def get_data_by_year(self, year):
        url = f'{self.base_url}{year}/index.html'
        return url
    
    def scrapping_data(self, url):
        page = requests.get(url)
                
        soup = BeautifulSoup(page.content, 'html.parser')

        # Obtención de datos del texto
        data_text = soup.get_text()

        # Encontrar números enteros en el texto
        data_values = [int(value) for value in re.findall(r'-?\d+', data_text)]

        # Dividimos los datos en sublistas para cada día
        day_length = 24
        all_day_data = [data_values[i:i+day_length] for i in range(26, len(data_values), day_length+1)]

        dataframe = pd.DataFrame(all_day_data, index=range(1,len(all_day_data)+1), columns=range(1,len(all_day_data[0])+1))
        return dataframe


In [5]:
YEAR = 2000
MONTH = 1
DAY = 20
data = Data()
url_by_month = data.get_data_by_month(YEAR, MONTH)
df1 = data.scrapping_data(url_by_month)


print(df1.head())

   1   2   3   4   5   6   7   8   9   10  ...  15  16  17  18  19  20  21  \
1 -46 -38 -38 -42 -45 -40 -38 -35 -38 -30  ... -22 -23 -21 -23 -24 -28 -25   
2 -24 -23 -25 -24 -26 -29 -22 -20 -21 -21  ... -16 -16 -18 -19 -12 -19 -19   
3 -17 -21 -21 -20 -23 -20 -23 -21 -19 -22  ... -19 -16 -18 -20 -17 -18 -15   
4 -10 -11 -14 -15 -16 -15 -14 -18 -20 -22  ... -19 -16 -14 -13 -16 -15 -17   
5  -9 -18 -28 -32 -28 -23 -23 -34 -32 -34  ... -24 -21 -23 -30 -29 -28 -26   

   22  23  24  
1 -27 -28 -28  
2 -17 -15 -14  
3 -12 -11 -11  
4 -15 -17 -10  
5 -29 -26 -19  

[5 rows x 24 columns]


In [8]:
print(df1.index())

TypeError: 'RangeIndex' object is not callable

In [11]:
import plotly.express as px

fig = px.line(df1, x=df1.index, y=df1.columns, template='plotly_dark')
fig.show()

In [12]:
fig = px.scatter(df1, x=df1.index, y=df1.columns, template='plotly_dark')
fig.show()

d:\Descargas\Anaconda\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [16]:
column_names = [f'{i}' for i in range(1,25)]
day_count = df1.shape[0] #numero de filas (dias)
row_names = [f'Día {i}' for i in range(1,day_count+1)]
# asigno nombres a las columnas y filas
df_copy = df1.copy()
df_copy.columns = column_names
df_copy.index = row_names
df_copy.head()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
Día 1,-46,-38,-38,-42,-45,-40,-38,-35,-38,-30,...,-22,-23,-21,-23,-24,-28,-25,-27,-28,-28
Día 2,-24,-23,-25,-24,-26,-29,-22,-20,-21,-21,...,-16,-16,-18,-19,-12,-19,-19,-17,-15,-14
Día 3,-17,-21,-21,-20,-23,-20,-23,-21,-19,-22,...,-19,-16,-18,-20,-17,-18,-15,-12,-11,-11
Día 4,-10,-11,-14,-15,-16,-15,-14,-18,-20,-22,...,-19,-16,-14,-13,-16,-15,-17,-15,-17,-10
Día 5,-9,-18,-28,-32,-28,-23,-23,-34,-32,-34,...,-24,-21,-23,-30,-29,-28,-26,-29,-26,-19


In [21]:
df_copy.iloc[0]

1    -46
2    -38
3    -38
4    -42
5    -45
6    -40
7    -38
8    -35
9    -38
10   -30
11   -24
12   -22
13   -22
14   -24
15   -22
16   -23
17   -21
18   -23
19   -24
20   -28
21   -25
22   -27
23   -28
24   -28
Name: Día 1, dtype: int64

In [25]:
# extraer cada fila de la tabla
df_2 = pd.DataFrame()
for i in range(len(df_copy)):
    row = df_copy.iloc[i]
    row.columns = [f'Día {i+1}']
    df_2 = pd.concat([df_2, row], axis=1)
df_2.head(10)

,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,...,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30,Día 31
1,-46,-24,-17,-10,-9,-19,-23,-13,-7,3,...,1,-97,-38,-36,-22,-27,-31,-29,-24,-18
2,-38,-23,-21,-11,-18,-20,-22,-12,-9,-1,...,7,-92,-38,-32,-20,-24,-41,-23,-22,-16
3,-38,-25,-21,-14,-28,-17,-23,-11,-10,-4,...,16,-96,-37,-33,-17,-17,-38,-23,-21,-22
4,-42,-24,-20,-15,-32,-15,-25,-10,-11,-6,...,12,-97,-39,-36,-24,-12,-37,-23,-19,-19
5,-45,-26,-23,-16,-28,-14,-30,-9,-8,-4,...,9,-79,-43,-41,-30,-13,-40,-27,-22,-18
6,-40,-29,-20,-15,-23,-18,-28,-10,-6,1,...,11,-65,-47,-39,-28,-12,-35,-28,-24,-18
7,-38,-22,-23,-14,-23,-17,-25,-12,-4,3,...,6,-66,-44,-39,-22,-1,-36,-29,-25,-20
8,-35,-20,-21,-18,-34,-14,-20,-11,-3,-1,...,1,-62,-43,-39,-17,5,-30,-22,-18,-19
9,-38,-21,-19,-20,-32,-11,-20,-9,-5,-1,...,8,-63,-46,-39,-16,8,-28,-28,-17,-18
10,-30,-21,-22,-22,-34,-17,-24,-12,-7,0,...,4,-69,-52,-40,-16,7,-36,-30,-18,-17


In [27]:
# convertir todas las columnas en una sola columna, crear una columna al lado que contiene que dia es 


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['Día 1', 'Día 2', 'Día 3', 'Día 4', 'Día 5', 'Día 6', 'Día 7', 'Día 8', 'Día 9', 'Día 10', 'Día 11', 'Día 12', 'Día 13', 'Día 14', 'Día 15', 'Día 16', 'Día 17', 'Día 18', 'Día 19', 'Día 20', 'Día 21', 'Día 22', 'Día 23', 'Día 24', 'Día 25', 'Día 26', 'Día 27', 'Día 28', 'Día 29', 'Día 30', 'Día 31'] but received: 31